In [38]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from prophet import Prophet
import pmdarima as pm

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

from sklearn.ensemble import RandomForestRegressor
from copy import deepcopy


from keras.models import Sequential
from keras.layers import Dense


import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger("cmdstanpy").disabled = True

In [39]:
mpl.rcParams['figure.figsize'] = (12, 8)
mpl.rcParams['axes.grid'] = False

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
colors = mpl.colormaps['coolwarm'].resampled(10)
colors = colors(np.linspace(0, 1, 10))

In [40]:
df = pd.read_csv("Anon_Cust_Order_Data.csv")
df.head()

,Order_ID,CustomerID,Date,State,City,ZIP,Product,Quantity,Price_Each
0,176558,1,4/19/19 8:46,TX,Dallas,75001,USB-C Charging Cable,2,11.95
1,176559,2,4/7/19 22:30,MA,Boston,2215,Bose SoundSport Headphones,1,99.99
2,176560,3,4/12/19 14:38,CA,Los Angeles,90001,Google Phone,1,600.00
3,176560,3,4/12/19 14:38,CA,Los Angeles,90001,Wired Headphones,1,11.99
4,176561,4,4/30/19 9:27,CA,Los Angeles,90001,Wired Headphones,1,11.99


## Time Series Forecasting

The stats library in Python has tools for building several other time-series prediction models - ARMA, ARIMA, and SARIMA - with just a few lines of code. 

Since all of these models are available in a single library, we can easily run many Python forecasting experiments using different models in the same script or notebook when conducting time series forecasting in Python. 

https://builtin.com/data-science/time-series-forecasting-python

### Experiment with different levels of prediction:
There is a tradeoff between granular prediction and speed of moel output. The goal of this algorithm is to find the best tradeoff between model prediction accuracy and cost of model (run time, resources). By presenting a range of outcomes, our client will have the ability to choose based on their business goals.

#### Level 1: Separate Regionally:

- Separate dataset by product and city.
- Train predictive machine learning algorithms

In [41]:
prods = df['Product'].unique()
cities = df['City'].unique()
total = len(prods) * len(cities)
print(prods, cities, total)

['USB-C Charging Cable' 'Bose SoundSport Headphones' 'Google Phone'
 'Wired Headphones' 'Macbook Pro Laptop' 'Lightning Charging Cable'
 '27in 4K Gaming Monitor' 'AA Batteries (4-pack)'
 'Apple Airpods Headphones' 'AAA Batteries (4-pack)' 'iPhone'
 'Flatscreen TV' '27in FHD Monitor' '20in Monitor' 'LG Dryer'
 'ThinkPad Laptop' 'Vareebadd Phone' 'LG Washing Machine'
 '34in Ultrawide Monitor'] ['Dallas' 'Boston' 'Los Angeles' 'San Francisco' 'Seattle' 'Atlanta'
 'New York City' 'Portland' 'Austin'] 171


#### Write programs for running several time-series predictive ML algorithms so we can compare results:

In [42]:
def ARIMA_model (train_df, test_df, order0, order1, order2):

    ARIMAmodel = ARIMA(train_df, order = (order0, order1, order2))
    ARIMAmodel = ARIMAmodel.fit()

    y_pred = ARIMAmodel.get_forecast(len(test_df.index))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test_df.index
    y_pred_out = y_pred_df["Predictions"] 
    
    #plt.plot(y_pred_out, color='Red', label = 'ARIMA Predictions')
    # plt.show()

    arima_rmse = np.sqrt(mean_squared_error(test_df["Quantity"].values, y_pred_df["Predictions"]))

    return arima_rmse

In [43]:
def SARIMAX_model (train_df, test_df, order0, order1, order2, order3):
    
    SARIMAXmodel = SARIMAX(train_df, seasonal_order=(order0, order1, order2, order3))
    SARIMAXmodel = SARIMAXmodel.fit(disp=False)

    y_pred = SARIMAXmodel.get_forecast(len(test_df.index))
    y_pred_df = y_pred.conf_int(alpha = 0.05) 
    y_pred_df["Predictions"] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
    y_pred_df.index = test_df.index
    y_pred_out = y_pred_df["Predictions"] 
   
    #plt.plot(y_pred_out, color='Blue', label = 'SARIMA Predictions')
    #plt.show()
    
    sarimax_rmse = np.sqrt(mean_squared_error(test_df["Quantity"].values, y_pred_df["Predictions"]))

    return sarimax_rmse

In [44]:
def run_prophet(p_df):

    p_df["Date"] = p_df["Date"].dt.to_period("W").dt.start_time
    
    p_df = p_df.groupby('Date', as_index=False).agg('sum')
    
    p_df.columns = ['ds', 'y']
    
    p_train = p_df[p_df.index <= 40]
    p_test = p_df[p_df.index > 40]
    
    p_model = Prophet()
    
    if p_train.shape[0] > 0 and p_test.shape[0] > 0:
        p_model.fit(p_train)
        p_forecast = p_model.predict(p_test)
        
        ## Calculate RMSE:
        y_actual = p_test['y']
        y_predicted = p_forecast['yhat']
    
        p_rmse = np.sqrt(mean_squared_error(y_actual, y_predicted))
    
    else:
         p_rmse = 1000 
    return p_rmse

In [45]:
def convert2matrix(dnn_arr, look_back):
    
    X = []
    Y = []

    for i in range(len(dnn_arr)-look_back):
        d = i+look_back
        X.append(dnn_arr[i:d, 0])
        Y.append(dnn_arr[d, 0])
    
    return np.array(X), np.array(Y)

In [46]:
def run_DNN (dnn_df):

    train_size = 40

    d_train, d_test = dnn_df.values[0:train_size,:], dnn_df.values[train_size:len(dnn_df.values),:]
    
    look_back = 4

    trainX, trainY = convert2matrix(d_train, look_back)
    testX, testY = convert2matrix(d_test, look_back)


    d_model= Sequential()
    d_model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    d_model.add(Dense(8, activation='relu'))
    d_model.add(Dense(1))
    d_model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    
    history = d_model.fit(trainX,trainY, epochs=10, batch_size=None, verbose=0, validation_data=(testX,testY),shuffle=False)

    #d_forecast = d_model.predict(testX)
    test_score = d_model.evaluate(testX, testY, verbose=0)
    

    ## Calculate RMSE:
    if len(test_score) >0:
        d_rmse = np.sqrt(test_score[1])
    else:
        d_rmse = 1000

    return d_rmse

In [47]:
class RandomForestARModel():
    """
    Autoregressive forecasting with Random Forests
    """
    
    def __init__(self, n_lags=1, max_depth = 3, n_estimators=1000, random_state = 123,
                 log_transform = False, first_differences = False, seasonal_differences = None):
        """
        Args:
            n_lags: Number of lagged features to consider in autoregressive model
            max_depth: Max depth for the forest's regression trees
            random_state: Random state to pass to random forest
            
            log_transform: Whether the input should be log-transformed
            first_differences: Whether the input should be singly differenced
            seasonal_differences: Seasonality to consider, if 'None' then no seasonality is presumed
        """
        
        self.n_lags = n_lags
        self.model = RandomForestRegressor(max_depth = max_depth, n_estimators = n_estimators, random_state = random_state)
        
        self.log_transform = log_transform
        self.first_differences = first_differences
        self.seasonal_differences = seasonal_differences
        
        
        
    def fit(self, y):
        """
        Args:
            y: training data (numpy array or pandas series/dataframe)
        """
        #enable pandas functions via dataframes
        y_df = pd.DataFrame(y)
        self.y_df = deepcopy(y_df)
        
        #apply transformations and store results for retransformations
        if self.log_transform:
            y_df = np.log(y_df)
            self.y_logged = deepcopy(y_df)
        
        if self.first_differences:
            y_df = y_df.diff().dropna()
            self.y_diffed = deepcopy(y_df)
        
        if self.seasonal_differences is not None:
            y_df = y_df.diff(self.seasonal_differences).dropna()
            self.y_diffed_seasonal = deepcopy(y_df)
        
        
        #get lagged features
        Xtrain = pd.concat([y_df.shift(t) for t in range(1,self.n_lags+1)],axis=1).dropna()
        self.Xtrain = Xtrain
        
        ytrain = y_df.loc[Xtrain.index,:]
        self.ytrain = ytrain

        self.model.fit(Xtrain.values,ytrain.values.reshape(-1))

    
    
    def sample_forecast(self, n_periods = 1, n_samples = 10000, random_seed =123):
        """
        Draw forecasting samples by randomly drawing from all trees in the forest per forecast period
        Args:
            n_periods: Ammount of periods to forecast
            n_samples: Number of samples to draw
            random_seed: Random seed for numpy
        """
        samples = self._perform_forecast(n_periods, n_samples, random_seed)
        output = self._retransform_forecast(samples, n_periods)
        
        return output
    
    
    
    def _perform_forecast(self, n_periods, n_samples, random_seed):
        """
        Forecast transformed observations
        Args:
            n_periods: Ammount of periods to forecast
            n_samples: Number of samples to draw
            random_seed: Random seed for numpy
        """
        samples = []
        
        np.random.seed(random_seed)
        for i in range(n_samples):
            #store lagged features for each period
            Xf = np.concatenate([self.Xtrain.iloc[-1,1:].values.reshape(1,-1),
                                 self.ytrain.iloc[-1].values.reshape(1,1)],1)

            forecasts = []

            for t in range(n_periods):
                tree = self.model.estimators_[np.random.randint(len(self.model.estimators_))]
                pred = tree.predict(Xf)[0]
                forecasts.append(pred)
                
                #update lagged features for next period
                Xf = np.concatenate([Xf[:,1:],np.array([[pred]])],1)
            
            samples.append(forecasts)
        
        return samples
    
    
    
    def _retransform_forecast(self, samples, n_periods):
        """
        Retransform forecast (re-difference and exponentiate)
        Args:
            samples: Forecast samples for retransformation
            n_periods: Ammount of periods to forecast
        """
        
        full_sample_tree = []

        for samp in samples:
            draw = np.array(samp)
            
            #retransform seasonal differencing
            if self.seasonal_differences is not None:
                result = list(self.y_diffed.iloc[-self.seasonal_differences:].values)
                for t in range(n_periods):
                    result.append(result[t]+draw[t])
                result = result[self.seasonal_differences:]
            else:
                result = []
                for t in range(n_periods):
                    result.append(draw[t])
            
            #retransform first differences
            y_for_add = self.y_logged.values[-1] if self.log_transform else self.y_df.values[-1]
            
            if self.first_differences:
                result = y_for_add + np.cumsum(result)
            
            #retransform log transformation
            if self.log_transform:
                result = np.exp(result)
            
            full_sample_tree.append(result.reshape(-1,1))

        return np.concatenate(full_sample_tree,1)

#### Compile models into a baseline program that will run all:

In [58]:
def model_selection_baseline(df):
    results = []
    a_results = []
    s_results = []
    p_results = []
    nnet_results = []
    rf_results = []
    
    for i in range(len(prods)):
        prod = prods[i]
        
        model_df = df[df['Product'] == prod]
    
        model_df['Date'] = pd.to_datetime(df['Date'])
        prophet_df = model_df[['Date','Quantity']]

        ## Group by Week:
        #model_df['week'] = model_df['Date'].dt.week  #.dt.week was depreciated
        model_df['week'] = model_df['Date'].dt.isocalendar().week
            
        grouped_df = model_df[['week','Quantity']]
        nnet_df = grouped_df.groupby(['week'], as_index = False).agg('sum')   ## Add in df formatted for NNET
        grouped_df = grouped_df.groupby(['week']).agg('sum')

            
        # Split into train / test set
            
        train = grouped_df[grouped_df.index <= 40]
        test = grouped_df[grouped_df.index > 40]
        y = train['Quantity']
        print("Y equals ", y.head() )
            
        model = pm.auto_arima(y, 
                  seasonal=False,
                  #trace=True,
                  trace=False,
                  error_action='ignore',  
                  suppress_warnings=True, 
                  stepwise=True)

        order0 = model.order[0]
        order1 = model.order[1]
        order2 = model.order[2]

        rmse = ARIMA_model(train, test, order0, order1, order2)
        a_results.append((prod, 'ARIMA('+','.join(map(str, model.order))+')', rmse))

        s_model = pm.auto_arima(y, 
                  seasonal=True,
                  m=12,
                  #trace=True,
                  trace=False,
                  error_action='ignore',  
                  suppress_warnings=True, 
                  stepwise=True)

        s_order0 = s_model.order[0]
        s_order1 = s_model.order[1]
        s_order2 = s_model.order[2]
        s_order3 = s_model.seasonal_order[3]
        
        # Run the model:
        s_rmse = SARIMAX_model(train, test, s_order0, s_order1, s_order2, s_order3)
        s_results.append((prod, 'SARIMAX('+','.join(map(str, s_model.seasonal_order))+')', s_rmse))


        ## PROPHET:
        p_rmse = run_prophet(prophet_df)
        p_results.append((prod, 'PROPHET', p_rmse))

        ## Deep Neural Net:
        d_rmse = run_DNN(nnet_df)
        nnet_results.append((prod, 'NNET', d_rmse))

        ## Random Forest:
        RandomForestmodel = RandomForestARModel(n_lags = 2, log_transform = True, first_differences = True, seasonal_differences = 12)
        RandomForestmodel.fit(y)

        y_pred = RandomForestmodel.sample_forecast(n_periods=len(test), n_samples=10000)
        means_forest = np.mean(y_pred,1)

        RandomForest_rmse = np.sqrt(mean_squared_error(test.values[:,0], means_forest))
        rf_results.append((prod, 'RandomForestARModel', RandomForest_rmse))



    for j in range(len(a_results)):
        if s_results[j][2] < a_results[j][2] and s_results[j][2] < p_results[j][2] and s_results[j][2] < nnet_results[j][2]:
            results.append(s_results[j])
        elif p_results[j][2] < a_results[j][2] and p_results[j][2] < s_results[j][2] and p_results[j][2] < nnet_results[j][2]:
            results.append(p_results[j])
        elif nnet_results[j][2] < a_results[j][2] and nnet_results[j][2] < s_results[j][2] and nnet_results[j][2] < p_results[j][2]:
            results.append(nnet_results[j])
        elif rf_results[j][2] < a_results[j][2] and rf_results[j][2] < s_results[j][2] and rf_results[j][2] < p_results[j][2]:
            results.append(rf_results[j])
        else:
            results.append(a_results[j])
            
    return results

In [56]:
## TESTING!

def TEST_model_selection_baseline(df):
    results = []
    a_results = []
    s_results = []
    p_results = []
    nnet_results = []
    rf_results = []
    
    for i in range(len(prods)):
        prod = prods[i]
        
        model_df = df[df['Product'] == prod]
    
        model_df['Date'] = pd.to_datetime(df['Date'])
        
        
        print("PRINTING HERE")
        display(model_df.head())
        print(model_df.info())
        
        
        prophet_df = model_df[['Date','Quantity']]

        ## Group by Week:
        
        model_df['week'] = model_df['Date'].dt.isocalendar().week
        
        #model_df['week'] = model_df['Date'].dt.week
            
        
        
        
        grouped_df = model_df[['week','Quantity']]
        nnet_df = grouped_df.groupby(['week'], as_index = False).agg('sum')   ## Add in df formatted for NNET
        grouped_df = grouped_df.groupby(['week']).agg('sum')

        '''
        # Split into train / test set
            
        train = grouped_df[grouped_df.index <= 40]
        test = grouped_df[grouped_df.index > 40]
        y = train['Quantity']
        print("Y equals ", y.head() )
            
        model = pm.auto_arima(y, 
                  seasonal=False,
                  #trace=True,
                  trace=False,
                  error_action='ignore',  
                  suppress_warnings=True, 
                  stepwise=True)

        order0 = model.order[0]
        order1 = model.order[1]
        order2 = model.order[2]

        rmse = ARIMA_model(train, test, order0, order1, order2)
        a_results.append((prod, 'ARIMA('+','.join(map(str, model.order))+')', rmse))

        s_model = pm.auto_arima(y, 
                  seasonal=True,
                  m=12,
                  #trace=True,
                  trace=False,
                  error_action='ignore',  
                  suppress_warnings=True, 
                  stepwise=True)

        s_order0 = s_model.order[0]
        s_order1 = s_model.order[1]
        s_order2 = s_model.order[2]
        s_order3 = s_model.seasonal_order[3]
        
        # Run the model:
        s_rmse = SARIMAX_model(train, test, s_order0, s_order1, s_order2, s_order3)
        s_results.append((prod, 'SARIMAX('+','.join(map(str, s_model.seasonal_order))+')', s_rmse))


        ## PROPHET:
        p_rmse = run_prophet(prophet_df)
        p_results.append((prod, 'PROPHET', p_rmse))

        ## Deep Neural Net:
        d_rmse = run_DNN(nnet_df)
        nnet_results.append((prod, 'NNET', d_rmse))

        ## Random Forest:
        RandomForestmodel = RandomForestARModel(n_lags = 2, log_transform = True, first_differences = True, seasonal_differences = 12)
        RandomForestmodel.fit(y)

        y_pred = RandomForestmodel.sample_forecast(n_periods=len(test), n_samples=10000)
        means_forest = np.mean(y_pred,1)

        RandomForest_rmse = np.sqrt(mean_squared_error(test.values[:,0], means_forest))
        rf_results.append((prod, 'RandomForestARModel', RandomForest_rmse))



    for j in range(len(a_results)):
        if s_results[j][2] < a_results[j][2] and s_results[j][2] < p_results[j][2] and s_results[j][2] < nnet_results[j][2]:
            results.append(s_results[j])
        elif p_results[j][2] < a_results[j][2] and p_results[j][2] < s_results[j][2] and p_results[j][2] < nnet_results[j][2]:
            results.append(p_results[j])
        elif nnet_results[j][2] < a_results[j][2] and nnet_results[j][2] < s_results[j][2] and nnet_results[j][2] < p_results[j][2]:
            results.append(nnet_results[j])
        elif rf_results[j][2] < a_results[j][2] and rf_results[j][2] < s_results[j][2] and rf_results[j][2] < p_results[j][2]:
            results.append(rf_results[j])
        else:
            results.append(a_results[j])
        '''            
    return results

#### Show RMSE tables by product:

In [59]:
def results_by_prod (product_list, result_list):
    prod_dict = {"Product":[], "Model": [],"RMSE": []}

    for i in range(len(product_list)):
        prod_name = product_list[i]

        for j in range(len(result_list)):  
            
            if result_list[j][0] == prod_name:
                prod_dict['Product'].append(result_list[j][0])
                prod_dict["Model"].append(result_list[j][1])
                prod_dict["RMSE"].append(result_list[j][2])
    
    return prod_dict

### Run Model Selection

In [60]:
#TEST_model_selection_baseline(df) 


In [61]:
baselines = model_selection_baseline(df)
baselines

Y equals  week
1    467
2    274
3    305
4    294
5    338
Name: Quantity, dtype: int64


2024-10-02 11:02:27.314813: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).